In [11]:
#
# 공식 문서의 import code ===========================================================================
#
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
import json
import copy
from keras.models import load_model
#
# =================================================================================================
#


#
# 파라미터 조정하는 부분 ================================================================================
#

batch_size = 100
num_classes = 576
epochs = 12
basePoint = [126.916810,37.548029]
squareSize=0.011

img_rows, img_cols = 24, 24

input_shape=(24,24,1)

#train :  test = r_train :  r_test 개수 비율
r_train=5
r_test=1

pNum=10

#
# ==================================================================================================
#   

In [12]:
#
# 학습 할 json 파일 열기 ===============================================================================
#

with open("./data/trainData.json","r") as js:
    jsonData=json.load(js)

#
# ==================================================================================================
#   

In [13]:
#
# json 파일을 이용해 x_train,y_train 만드는 함수==========================================================
#
def MakeTrainSet(jsonData):
    
    x_train=list()
    y_train=list()
    
    # 24*24 중 1개의 타일 변의 크기
    dl = squareSize / img_rows

    # basePoint 사각형 내에 있는 데이터만 거르기
    verifyJsonData=list()
    for x in jsonData:
        if basePoint[0]<= float(x['longitude']) <basePoint[0]+squareSize:
            if basePoint[1] <= float(x['latitude']) < basePoint[1]+squareSize:
                verifyJsonData.append(x)

    # 검증된 데이터셋을 [x좌표,y좌표,번호] 형식으로 만들기
    locationSet=list()
    for x in verifyJsonData:
        idx,idy=0,0
        for _idx in range(1,img_rows+1):
            if basePoint[0]+_idx*dl > float(x['longitude']):
                idx = _idx-1
                break
        for _idy in range(1,img_rows+1):
            if basePoint[1]+_idy*dl > float(x['latitude']):
                idy = _idy-1
                break
        locationSet.append([idx,idy,idx+img_rows*idy])

    # pNum의 개수만큼 점의 개수를 가지는 이미지를 x_train로 만들고 y_train으로 정답번호 주기
    tmpmap = [ [0 for _ in range(img_rows)] for _ in range(img_rows)]
    for i in range(pNum):
        tmpmap[locationSet[i][0]][locationSet[i][1]]+=1
    for i in range(pNum+1,len(locationSet)):
        x_train.append(copy.deepcopy(tmpmap))
        y_train.append(locationSet[i][2])
        tmpmap[locationSet[i][0]][locationSet[i][1]]+=1
        tmpmap[locationSet[i-pNum+1][0]][locationSet[i-pNum+1][1]]-=1
        
        
    # np array 로 만들기
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    
    # train : test = r_train : r_test
    line1 = (x_train.shape[0]//(r_train+r_test))*r_train

    x_test = x_train[line1:]
    x_train = x_train[0:line1]

    y_test = y_train[line1:]
    y_train = y_train[0:line1]
    
    #혹시 케라스 이미지 포멧이 채널이 first인경우는 바꿔준다
    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)
        
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= pNum
    x_test /= pNum
    
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    return x_train,x_test,y_train,y_test
    
#
# ==================================================================================================
# 

In [14]:
#
# 데이터 전처리 하는 code 및 일부 데이터 확인 ===============================================================
#

x_train,x_test,y_train,y_test = MakeTrainSet(jsonData)

#
# ==================================================================================================
# 

In [15]:
#
# 모델 정의 및 학습=====================================================================================
#

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save('../keraslocationpredict.h5')
#
# ==================================================================================================
# 

Train on 21060 samples, validate on 4215 samples
Epoch 1/12
21060/21060 [==============================] - 11s 521us/step - loss: 3.2218 - acc: 0.3028 - val_loss: 2.6730 - val_acc: 0.3770
Epoch 2/12
21060/21060 [==============================] - 10s 484us/step - loss: 1.9200 - acc: 0.4501 - val_loss: 2.1908 - val_acc: 0.4052
Epoch 3/12
21060/21060 [==============================] - 10s 492us/step - loss: 1.7414 - acc: 0.4715 - val_loss: 1.9822 - val_acc: 0.4365
Epoch 4/12
21060/21060 [==============================] - 10s 488us/step - loss: 1.6609 - acc: 0.4813 - val_loss: 2.0282 - val_acc: 0.4482
Epoch 5/12
21060/21060 [==============================] - 10s 483us/step - loss: 1.6192 - acc: 0.4888 - val_loss: 1.9986 - val_acc: 0.4550
Epoch 6/12
21060/21060 [==============================] - 10s 466us/step - loss: 1.5869 - acc: 0.4914 - val_loss: 2.0374 - val_acc: 0.3924
Epoch 7/12
21060/21060 [==============================] - 9s 430us/step - loss: 1.5648 - acc: 0.4943 - val_loss: 1.94